In [1]:
import pandas as pd
from binance.client import Client
from datetime import datetime 

import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout , LSTM , CuDNNLSTM
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt
import numpy as np
#import tensorflow.compat.v2 as tf

import tensorflow_datasets as tfds
from sklearn import preprocessing
from collections import deque
import random
import time

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout , LSTM , CuDNNLSTM , BatchNormalization
# Batchnormalization : so as to dont only normalize your input data , normalize your data layer through layer 
from tensorflow.keras.callbacks import TensorBoard , ModelCheckpoint

Parameters

In [2]:
seq = 10
future_period_predict = 3
ratio_to_predict = 'LTCUSDT'
bachsize = 64
name = f"{seq}-seq-{future_period_predict}-pred-{ratio_to_predict}"

Fetch data

In [3]:
api_key = 'umXNh30LxCf9EaAVDFX8yBHBXQ8vyyLUbvV0JxY19kAdnrYweqDCCqLaHs3QJmJ9'
api_secret = '1so1diWCzjuJyh5dUG4xQXJzDSHAWhhFRpuD5A3aQs5CfgQNEnDEewTmUV2YRZXI'

In [4]:
client = Client(api_key, api_secret)

In [5]:
cryptos = ['BTCUSDT' , 'LTCUSDT' , 'ETHUSDT']
main_df = []

for crypto in cryptos:
    candles = client.get_klines(symbol=crypto , interval=Client.KLINE_INTERVAL_1DAY)

    candles_df = pd.DataFrame(candles)

    candles_df_date = candles_df[0]

    final_date = []

    for time in candles_df_date.unique():
        dates = datetime.fromtimestamp(int(time/1000))
        final_date.append(dates)

    candles_df.pop(0)
    candles_df.pop(11)

    final_date = pd.DataFrame(final_date)

    final_date.columns = ['date']

    final_df = candles_df.join(final_date)

    final_df.set_index('date' , inplace=True)

    final_df.columns = ['open' , 'high' , 'low' ,'close' ,'volume' ,'close_time' ,'asset_value' , 'trades' , 'buy_base_asset_v' , 'buy_quote_asset_v']
    final_df = final_df[['close' , 'volume']]
    
    final_df.rename(columns={'close': f"{crypto}_close" , 'volume': f"{crypto}_volume"} , inplace=True)
    
    main_df.append(final_df)


In [6]:
df = pd.concat(main_df , axis=1)

In [7]:
df.head()

,BTCUSDT_close,BTCUSDT_volume,LTCUSDT_close,LTCUSDT_volume,ETHUSDT_close,ETHUSDT_volume
date,,,,,,
2018-09-09 21:00:00,6312.00000000,32439.45643200,54.34000000,226840.05414000,197.15000000,418489.86663000
2018-09-10 21:00:00,6294.91000000,33104.88859000,52.04000000,239830.95074000,185.18000000,476745.79216000
2018-09-11 21:00:00,6338.62000000,31433.64593900,51.69000000,276375.64336000,183.12000000,504047.79165000
2018-09-12 21:00:00,6487.38000000,40465.06310700,54.44000000,232884.22924000,210.68000000,739503.34051000
2018-09-13 21:00:00,6476.63000000,37173.33927700,56.24000000,279002.97525000,208.82000000,833945.82573000


In [8]:
df['BTCUSDT_close'] = df['BTCUSDT_close'].astype(str).astype(float)
df['BTCUSDT_volume'] = df['BTCUSDT_volume'].astype(str).astype(float)
df['LTCUSDT_close'] = df['LTCUSDT_close'].astype(str).astype(float)
df['LTCUSDT_volume'] = df['LTCUSDT_volume'].astype(str).astype(float)
df['ETHUSDT_close'] = df['ETHUSDT_close'].astype(str).astype(float)
df['ETHUSDT_volume'] = df['ETHUSDT_volume'].astype(str).astype(float)
#df['BCHUSDT_close'] = df['BCHUSDT_close'].astype(str).astype(float)
#df['BCHUSDT_volume'] = df['BCHUSDT_volume'].astype(str).astype(float)

In [9]:
# Bitcoin cash have same values
#bch = df['BCHUSDT_close']
#bch.plot()

Build strategy : if the future value expected is bigger than the current price , buy

In [10]:
def classify(current , future):
    if float(future) > float(current):
        return 1
    else:
        return 0

In [11]:
df['future'] = df[f"{ratio_to_predict}_close"].shift(-future_period_predict)

In [12]:
# Me quedo con las series q tienen datos completas
#df = df[df.index > datetime(2019 , 11 , 27)]

In [13]:
df.head()

,BTCUSDT_close,BTCUSDT_volume,LTCUSDT_close,LTCUSDT_volume,ETHUSDT_close,ETHUSDT_volume,future
date,,,,,,,
2018-09-09 21:00:00,6312.00,32439.456432,54.34,226840.05414,197.15,418489.86663,54.44
2018-09-10 21:00:00,6294.91,33104.888590,52.04,239830.95074,185.18,476745.79216,56.24
2018-09-11 21:00:00,6338.62,31433.645939,51.69,276375.64336,183.12,504047.79165,56.22
2018-09-12 21:00:00,6487.38,40465.063107,54.44,232884.22924,210.68,739503.34051,56.75
2018-09-13 21:00:00,6476.63,37173.339277,56.24,279002.97525,208.82,833945.82573,52.07


In [14]:
df['target'] = list(map(classify ,df[f"{ratio_to_predict}_close"] , df['future'] ))

In [15]:
last_5pct = round(df.shape[0]*float(0.05))

In [16]:
train = df[:(df.shape[0] - last_5pct)]
test = df[(df.shape[0] - last_5pct):]

In [17]:
# Idea is to calculate with the previous "X" days 

def preprocess_df(df):
    df = df.drop("future", 1)  # don't need this anymore.

    for col in df.columns:  # go through all of the columns
        if col != "target":  # normalize all ... except for the target itself!
            df[col] = df[col].pct_change()  # pct change "normalizes" the different currencies (each crypto coin has vastly diff values, we're really more interested in the other coin's movements)
            df.dropna(inplace=True)  # remove the nas created by pct_change
            df[col] = preprocessing.scale(df[col].values)  # scale between 0 and 1.

    df.dropna(inplace=True)  # cleanup again... jic.


    sequential_data = []  # this is a list that will CONTAIN the sequences
    prev_days = deque(maxlen=10)  # These will be our actual sequences. They are made with deque, which keeps the maximum length by popping out older values as new ones come in

    for i in df.values:  # iterate over the values
        prev_days.append([n for n in i[:-1]])  # store all but the target
        if len(prev_days) == 10:  # make sure we have 60 sequences!
            sequential_data.append([np.array(prev_days), i[-1]])  # append those bad boys!

    random.shuffle(sequential_data)  # shuffle for good measure.

    buys = []  # list that will store our buy sequences and targets
    sells = []  # list that will store our sell sequences and targets

    for seq, target in sequential_data:  # iterate over the sequential data
        if target == 0:  # if it's a "not buy"
            sells.append([seq, target])  # append to sells list
        elif target == 1:  # otherwise if the target is a 1...
            buys.append([seq, target])  # it's a buy!

    random.shuffle(buys)  # shuffle the buys
    random.shuffle(sells)  # shuffle the sells!

    lower = min(len(buys), len(sells))  # what's the shorter length?

    buys = buys[:lower]  # make sure both lists are only up to the shortest length.
    sells = sells[:lower]  # make sure both lists are only up to the shortest length.

    sequential_data = buys+sells  # add them together
    random.shuffle(sequential_data)  # another shuffle, so the model doesn't get confused with all 1 class then the other.

    X = []
    y = []

    for seq, target in sequential_data:  # going over our new sequential data
        X.append(seq)  # X is the sequences
        y.append(target)  # y is the targets/labels (buys vs sell/notbuy)

    return np.array(X), y  # return X and y...and make X a numpy array!

In [18]:
train.shape

(475, 8)

In [19]:
train_x , train_y = preprocess_df(train)
test_x , test_y = preprocess_df(test)

Specify model

In [20]:
#model = Sequential()
#model.add(LSTM(128 , input_shape=(train_x.shape[:1]) , activation = 'sigmoid' , return_sequences=True))
#model.add(Dropout(0.2))
#model.add(BatchNormalization())
model = Sequential()
model.add(LSTM(128, input_shape=(train_x.shape[1:]),activation='relu' ,return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(128 , activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32 , activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))


opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

# Compile model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

#tensorboard = TensorBoard(log_dir="logs/{}".format(name))

#filepath = "RNN_Final-{epoch:02d}-{val_acc:.3f}"  # unique file name that will include the epoch and the validation acc for that epoch
#checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')) # saves only the best ones

# Train model
model.fit(
    train_x, train_y,
    batch_size=bachsize,
    epochs=10,
    validation_data=(test_x, test_y)
   # callbacks=[tensorboard, checkpoint],
)

# Score model
#score = model.evaluate(test_x, test_y, verbose=0)
#print('Test loss:', score[0])
#print('Test accuracy:', score[1])
# Save model
#model.save("models/{}".format(NAME))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 426 samples, validate on 10 samples
Epoch 1/10
426/426 [==============================] - 2s 5ms/sample - loss: 0.9194 - acc: 0.5023 - val_loss: 0.6949 - val_acc: 0.5000
Epoch 2/10
426/426 [==============================] - 0s 874us/sample - loss: 0.8034 - acc: 0.5305 - val_loss: 0.6950 - val_acc: 0.5000
Epoch 3/10
426/426 [==============================] - 0s 703us/sample - loss: 0.7912 - acc: 0.5540 - val_loss: 0.6932 - val_acc: 0.5000
Epoch 4/10
426/426 [==============================] - 0s 923us/sample - loss: 0.7490 - acc: 0.5892 - val_loss: 0.6936 - val_acc: 0.5000
Epoch 5/10
426/426 [==============================] - 0s 841us/sample - loss: 0.6830 - acc: 0.5986 - val_loss: 0.6933 - val_acc: 0.5000
Epoch 6/10
426/426 [==============================] - 1s 1ms/sample - loss: 0.7159 - acc: 0.6080 - val